In [1]:
import os 
import sys 
sys.path.append("../training")
import pickle
import copy
import time

import hls4ml 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, auc, roc_curve

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

# Local imports
from save_data import process_data
from threshold import ThresholdModel

np.random.seed(0)

2024-04-21 16:05:03.569252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 16:05:03.673848: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-21 16:05:03.673870: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-21 16:05:04.147445: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'


/data/aelabd/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


In [2]:
START_WINDOW = 285
END_WINDOW = 385

# START_WINDOW = 150
# END_WINDOW = 350

# START_WINDOW = 150
# END_WINDOW = 550

# START_WINDOW = 0
# END_WINDOW = 770


!mkdir -p models

DATA_IN_DIR = f'../data/raw-data'
DATA_OUT_DIR = f'../data/qick_data/{START_WINDOW}_{END_WINDOW}'
MODEL_DIR = f'models/model_{START_WINDOW}_{END_WINDOW}'

# convert raw ADC data into npy files 
if os.path.exists(f'{DATA_OUT_DIR}/X_train.npy') == False:
    process_data(
        start_window = START_WINDOW, 
        end_window = END_WINDOW, 
        data_in = DATA_IN_DIR,
        data_out = DATA_OUT_DIR
    )

In [3]:
# load data
X_train_val = np.load(os.path.join(DATA_OUT_DIR, 'X_train.npy'))
X_test = np.load(os.path.join(DATA_OUT_DIR, 'X_test.npy'))    
y_train_val = np.load(os.path.join(DATA_OUT_DIR, 'y_train.npy'))
y_test = np.load(os.path.join(DATA_OUT_DIR, 'y_test.npy'))

# y_train_val = one_hot_encode(y_train_val) # 1xN binary --> 2xN one-hot-encoded
# y_test = one_hot_encode(y_test)

print("Training:")
print("\tSize:", len(X_train_val))
print("\tSample Shape:", len(X_train_val[0]))
print("\tMean:", X_train_val.mean())
print("\tStd. Dev.:", X_train_val.std())

print("Testing:")
print("\tSize:", len(X_test))
print("\tSample Shape:", len(X_test[0]))
print("\tSample Shape:", X_test.mean())
print("\tStd. Dev.:", X_test.std())

# X_train_val[0] = I, Q timeseries over (END_WINDOW - START_WINDOW) timesteps
assert len(X_train_val[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"
assert len(X_test[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"

Training:
	Size: 909000
	Sample Shape: 200
	Mean: -28.26868580858086
	Std. Dev.: 1757.307338388309
Testing:
	Size: 101000
	Sample Shape: 200
	Sample Shape: -21.936618267326732
	Std. Dev.: 1755.8698046711418


## Construct a model 

In [4]:
VALIDATION_SPLIT = 0.3
CHECKPOINT_FILENAME = os.path.join(MODEL_DIR, 'threshold_model.json')
INPUT_SHAPE = (len(X_train_val[0]),)
print(INPUT_SHAPE)

model = ThresholdModel()

(200,)


## Train the model 

In [5]:
tv_shuffler = np.random.permutation(X_train_val.shape[0])

X_train_val = X_train_val[tv_shuffler]
y_train_val = y_train_val[tv_shuffler]

X_train = X_train_val[:int((1-VALIDATION_SPLIT)*X_train_val.shape[0])]
y_train = y_train_val[:int((1-VALIDATION_SPLIT)*X_train_val.shape[0])]

model.fit(X_train, y_train)
model.save(CHECKPOINT_FILENAME)

Important: Pruning layers must be removed before saving to disk. 

In [6]:
y_pred = model.predict(X_test)
print("Threshold model  accuracy: {}".format(accuracy_score(y_test, y_pred)))

Threshold model  accuracy: 0.8785346534653465


## Check performance

In [7]:
model = ThresholdModel()
model.load(CHECKPOINT_FILENAME)
y_pred = model.predict(X_test)
print("Threshold model  accuracy: {}".format(accuracy_score(y_test, y_pred)))

Threshold model  accuracy: 0.8785346534653465
